In [111]:
import nltk 
import pandas as pd 
import numpy as np 
import re # regular expression 
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer 
from tensorflow.keras.preprocessing.text import Tokenizer   ## text to numerical representation 
from tensorflow.keras.utils import pad_sequences   # to equalize the length of sequences 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import SimpleRNN, Dense,LSTM 
print("Libraries imported!")

Libraries imported!


In [2]:
# nltk.download() 

In [3]:
with open("SMSSpamCollection.txt") as file : 
    content = file.readlines()

In [4]:
x = []
y = []
for email in content:
    email_with_label = email.split("\t")
    if len(email_with_label) == 2:
        x.append(email_with_label[1][:-1])
        y.append(email_with_label[0])

In [5]:
### dataframe 
df = pd.DataFrame({"email_message":x,"labels":y}) 
df.head(5)

,email_message,labels
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5574 entries, 0 to 5573
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   email_message  5574 non-null   object
 1   labels         5574 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [7]:
df['labels'].value_counts()

labels
ham     4827
spam     747
Name: count, dtype: int64

In [8]:
ham = df[df['labels'] == "ham"]
spam = df[df['labels'] == "spam"]

In [9]:
spam.shape , ham.shape 

((747, 2), (4827, 2))

In [10]:
HAM = ham.sample(1000)

In [11]:
HAM.shape 

(1000, 2)

In [12]:
df2 = pd.concat([HAM,spam])
df2.head(5)
## balance dataset 

,email_message,labels
2908,ALRITE,ham
1273,Ok...,ham
4462,Thanks again for your reply today. When is ur ...,ham
1383,Its ok my arm is feeling weak cuz i got a shot...,ham
2608,:-) yeah! Lol. Luckily i didn't have a starrin...,ham


In [13]:
df2.shape   # 1000 <== ham , 747 spam 

(1747, 2)

In [19]:
df2.reset_index(drop=True,inplace=True)

In [27]:
## text cleaning , unnecssary 
## text ==> numerical 
## 1. a-z A-Z 
df2['email_message'][300]

'Do u noe how 2 send files between 2 computers?'

In [56]:
## Text cleaning
stemming = PorterStemmer()
x = []
stopwords_ls =  stopwords.words('english')
for message in df2['email_message']:
    clean_message = re.sub("[^a-zA-z ]","",message)  # remove the un-necessary characters from the message 
    lower_clean_message = clean_message.lower()  # convert into lower case 
    ls_of_words = nltk.word_tokenize(lower_clean_message)  # divide the sentence into words
    ls_of_words_withoutstopwords = [word for word in ls_of_words if word not in stopwords_ls]  # remove stopwords
    ls_of_stemmed_words = [stemming.stem(word) for word in ls_of_words_withoutstopwords]  # applied stemming 
    clean_message = " ".join(ls_of_stemmed_words)
    x.append(clean_message)

In [53]:
# x[3].split() ### testing  
stopwords_ls =  stopwords.words('english')
single_message = nltk.word_tokenize(df2['email_message'][3])
print(single_message)
new_ls = []
for word in single_message:
    if word not in stopwords_ls:
        new_ls.append(word) 
new_ls

['Its', 'ok', 'my', 'arm', 'is', 'feeling', 'weak', 'cuz', 'i', 'got', 'a', 'shot', 'so', 'we', 'can', 'go', 'another', 'time']


['Its',
 'ok',
 'arm',
 'feeling',
 'weak',
 'cuz',
 'got',
 'shot',
 'go',
 'another',
 'time']

In [55]:
ls = ['Its',
 'ok',
 'arm',
 'feeling',
 'weak',
 'cuz',
 'got',
 'shot',
 'go',
 'another',
 'time']
" ".join(ls)

'Its ok arm feeling weak cuz got shot go another time'

In [62]:
df2['email_message'][259]   # message before cleaning

"Yeah you should. I think you can use your gt atm now to register. Not sure but if there's anyway i can help let me know. But when you do be sure you are ready."

In [63]:
x[259]   # m3ssage after cl3aning

'yeah think use gt atm regist sure there anyway help let know sure readi'

In [66]:
### Text to numerical representation 
tokenizer = Tokenizer(oov_token="<nothing>") 
tokenizer.fit_on_texts(x)

In [69]:
# tokenizer.word_index 
sequence = tokenizer.texts_to_sequences(x)

In [78]:
len(sequence[2])


26

In [82]:
len(x[2].split())

26

In [86]:
temp_ls = []
for message in sequence :
    temp_ls.append(len(message))
print(max(temp_ls))
# or   
print(max(list(map(len,sequence))))


80
80


In [93]:
sequences = pad_sequences(sequences=sequence,maxlen=80)

In [100]:
Y = np.array(df2['labels'].map({'ham':0,'spam':1})) 


In [102]:
sequences.shape , Y.shape 

((1747, 80), (1747,))

In [103]:
# sequences  ===>  X  
# y          ===>  y   

# training and testins split  
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test = train_test_split( sequences,Y,test_size=0.15,random_state=32)

In [ ]:
### Data cleaning 
### data processing 
### data transform 
### numerical representation 
### training testing split  
### Algorithm or model define  
### algorithm or model train 
### prediction 
### 

In [118]:
## RNN model define 
RNN_model = Sequential()
RNN_model.add(SimpleRNN(64,input_shape=(80,1),return_sequences=False))
RNN_model.add(Dense(1,activation='sigmoid'))

## compiling the model
RNN_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
RNN_model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn_5 (SimpleRNN)             │ (None, 64)                  │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,289 (16.75 KB)

 Trainable params: 4,289 (16.75 KB)

 Non-trainable params: 0 (0.00 B)

In [119]:
### train algorithm 
history = RNN_model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.5942 - loss: 0.6776 - val_accuracy: 0.5970 - val_loss: 0.6682
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.6040 - loss: 0.6647 - val_accuracy: 0.5741 - val_loss: 0.6638
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6123 - loss: 0.6650 - val_accuracy: 0.6008 - val_loss: 0.6612
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.5831 - loss: 0.6638 - val_accuracy: 0.6084 - val_loss: 0.6585
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.5886 - loss: 0.6663 - val_accuracy: 0.6084 - val_loss: 0.6555
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6222 - loss: 0.6531 - val_accuracy: 0.6160 - val_loss: 0.6576
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.6231 - loss: 0.6516 - val_accuracy: 0.6122 - val_loss: 0.6538
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.6324 - loss: 0.6458 - val_accuracy: 0.6160 - v

In [124]:
# LSTM model Define the model
lstm_model = Sequential([
    LSTM(units=64, input_shape=(80, 1), return_sequences=False),
    Dense(units=1, activation='sigmoid')
])
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_9 (LSTM)                        │ (None, 64)                  │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,961 (66.25 KB)

 Trainable params: 16,961 (66.25 KB)

 Non-trainable params: 0 (0.00 B)

In [126]:
history2 = lstm_model.fit(x_train,y_train, epochs=7,validation_data=(x_test,y_test))


Epoch 1/7
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.8586 - loss: 0.3532 - val_accuracy: 0.8099 - val_loss: 0.4079
Epoch 2/7
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.8526 - loss: 0.3369 - val_accuracy: 0.8175 - val_loss: 0.4011
Epoch 3/7
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.8599 - loss: 0.3297 - val_accuracy: 0.8251 - val_loss: 0.3951
Epoch 4/7
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.8738 - loss: 0.3043 - val_accuracy: 0.8137 - val_loss: 0.3995
Epoch 5/7
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.8739 - loss: 0.3101 - val_accuracy: 0.8213 - val_loss: 0.3980
Epoch 6/7
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.8664 - loss: 0.3289 - val_accuracy: 0.8175 - val_loss: 0.4022
Epoch 7/7
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.8599 - loss: 0.3322 - val_accuracy: 0.8213 - val_loss: 0.3898


In [130]:
### Machine learning algorithm 
from sklearn.naive_bayes import BernoulliNB   ## text classification 
bayes = BernoulliNB()
bayes.fit(x_train,y_train)


print("score : ")
print("Training score : ",bayes.score(x_train,y_train))
print("Training score : ",bayes.score(x_test,y_test))


score : 
Training score :  0.7998652291105122
Training score :  0.7984790874524715


In [131]:
### Machine learning algorithm 
from sklearn.naive_bayes import MultinomialNB   ## text classification 
bayes2 = MultinomialNB()
bayes2.fit(x_train,y_train)


print("score : ")
print("Training score : ",bayes2.score(x_train,y_train))
print("Training score : ",bayes2.score(x_test,y_test))


score : 
Training score :  0.7392183288409704
Training score :  0.7072243346007605


In [132]:
### prediction 
pred = lstm_model.predict(x_test)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step


In [137]:
predictions = ((pred > 0.5 ).astype(int)).ravel()

In [139]:
compare_df = pd.DataFrame({'actual':y_test,'prediction':predictions})

In [142]:
# compare_df.head(40)
from sklearn.metrics import confusion_matrix,classification_report 


In [143]:
confusion_matrix(compare_df['actual'],compare_df['prediction'])

array([[127,  18],
       [ 29,  89]], dtype=int64)

In [145]:
print(classification_report(compare_df['actual'],compare_df['prediction']))

              precision    recall  f1-score   support

           0       0.81      0.88      0.84       145
           1       0.83      0.75      0.79       118

    accuracy                           0.82       263
   macro avg       0.82      0.82      0.82       263
weighted avg       0.82      0.82      0.82       263

